In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bank-marketing-response-predict/submission.csv
/kaggle/input/bank-marketing-response-predict/test_label.csv
/kaggle/input/bank-marketing-response-predict/train.csv
/kaggle/input/bank-marketing-response-predict/test.csv


# There are 10 Questions for EDA in train.csv

In [2]:
df = pd.read_csv('/kaggle/input/bank-marketing-response-predict/train.csv')
df.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


# Question1
- 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 인원을 가진 나이대는? (0-9 : 0 , 10-19 : 10)
- Marketing Response When converting the age of customers to a 10-year-old unit, which age group has the largest number of people? (0-9 : 0 , 10-19 : 10)

In [3]:
df['ages'] = (df['age'] // 10) * 10
df.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y,ages
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no,20
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no,20
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no,30
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no,40
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no,20


In [4]:
df['ages'].value_counts().sort_values().index[-1]

30

# Question2
- 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 나이대 구간의 인원은 몇명인가?
- When converting the age of marketing response customers to a 10-year unit, what is the number of people in the largest age section?

In [5]:
df['ages'].value_counts().sort_values().iloc[-1]

5056

# Question3
- 나이가 25살 이상 29살 미만인 응답 고객들중 housing컬럼의 값이 yes인 고객의 수는?
- The number of respondents who are greater than or equal to 25 years old and under 29 years old with a value of 'yes' in 'housing' column


In [6]:
len(df.loc[((df['age'] >= 25) & (df['age'] < 29) & (df['housing'] == 'yes'))])

504

# Question4
- numeric한 값을 가지지 않은 컬럼들중 unique한 값을 가장 많이 가지는 컬럼은?
- Which column has the most unique value among columns that do not have numeric values?

In [7]:
df01 = df.select_dtypes(include=object)
df01.head()

,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,technician,single,tertiary,no,no,no,cellular,may,unknown,no
1,services,single,secondary,no,yes,yes,unknown,jun,unknown,no
2,management,single,secondary,no,no,no,cellular,aug,unknown,no
3,technician,married,unknown,no,yes,no,cellular,jul,unknown,no
4,admin.,single,secondary,no,no,yes,unknown,sep,unknown,no


In [8]:
list01 = []
for x in df01.columns:
    list01.append([x, df01[x].nunique()])

list01

[['job', 12],
 ['marital', 3],
 ['education', 4],
 ['default', 2],
 ['housing', 2],
 ['loan', 2],
 ['contact', 3],
 ['month', 12],
 ['poutcome', 4],
 ['y', 2]]

In [9]:
pd.DataFrame(list01, columns=['name', 'size']).sort_values('size', ascending=False).iloc[:2, :]

,name,size
0,job,12
7,month,12


# Question5
- balance 컬럼값들의 평균값 이상을 가지는 데이터를 ID값을 기준으로 내림차순 정렬했을때 상위 100개 데이터의 balance값의 평균은?
- What is the average of the balance values of the top 100 data when data having more than the average value of the balance column are sorted in descending order based on the ID value?

In [10]:
df.loc[(df['balance'] > df['balance'].mean())].sort_values('ID', ascending=False).head(100)['balance'].mean()

3473.73

# Question6
- 가장 많은 광고를 집행했던 날짜는 언제인가? (데이터 그대로 일(숫자),달(영문)으로 표기)
- When did you run the most ads?

In [11]:
df.groupby(['day', 'month']).size().to_frame('ad').sort_values('ad', ascending=False).index[0]

(15, 'may')

# Question7
- 데이터의 job이 unknown 상태인 고객들의 age 컬럼 값의 정규성을 검정하고자 한다. 샤피로 검정의 p-value값을 구하여라
- It is intended to test the normality of the age column value of customers whose data job is unknown. Find the p-value of the Shapiro test

In [12]:
df01 = df.loc[df['job'] == 'unknown']['age']
df01.head()

25     52
86     55
135    46
237    54
409    45
Name: age, dtype: int64

In [13]:
from scipy import stats
stats.shapiro(df01)[1]

0.1961131989955902

# Question8 
- age와 balance의 상관계수를 구하여라
- Find the correlation coefficient between age and balance columns

In [14]:
df[['age', 'balance']].corr().iloc[0,1]

0.10198734763981462

# Question9
- y 변수와 education 변수는 독립인지 카이제곱검정을 통해 확인하려한다. p-value값을 출력하라
- It will check whether the y variable and the education variable are independent through a chi-square test. Print the p-value value

In [15]:
df01 = pd.crosstab(df.y,df.education)
import scipy.stats as stats
a, b, c, d = stats.chi2_contingency(df01)
b

7.901201277473551e-29

# Question10
- 각 job에 따라 divorced/married 인원의 비율을 확인 했을 때 그 값이 가장 높은 값은?
- When checking the proportion of distributed/married people according to each job, what is the highest ratio?

In [16]:
df00 = df.groupby(['job', 'marital'], as_index=False).size()
df01 = df00.loc[(df00['marital'] == 'married')]
df02 = df00.loc[(df00['marital'] == 'divorced')]
df03 = pd.merge(df01, df02, on='job')
df03['ratio'] = df03['size_y'] / df03['size_x']
df03.sort_values('ratio')['ratio'].iloc[-1]

0.2831050228310502

# modeling

In [17]:
train =pd.read_csv('/kaggle/input/bank-marketing-response-predict/train.csv')
test =pd.read_csv('/kaggle/input/bank-marketing-response-predict/test.csv')
submission =pd.read_csv('/kaggle/input/bank-marketing-response-predict/submission.csv')

# predict "y" columns, "yes" if true, metrics to ROC score

In [18]:
display(train.head())
display(test.head())
display(submission.head())

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome
0,53608,32,management,single,tertiary,no,12569,no,no,cellular,1,jul,2,295,2,success
1,51055,25,services,single,secondary,no,801,no,no,cellular,5,jun,2,-1,0,unknown
2,52573,46,blue-collar,married,secondary,no,1728,yes,no,unknown,26,may,2,-1,0,unknown
3,50458,39,management,divorced,secondary,no,51,no,no,unknown,17,jun,2,-1,0,unknown
4,52272,31,services,single,tertiary,no,1626,no,no,unknown,31,jul,1,-1,0,unknown


,ID,predict
0,53608,0.0
1,51055,0.0
2,52573,0.0
3,50458,0.0
4,52272,0.0


In [19]:
display(train.info())
display(test.info())
display(submission.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12870 entries, 0 to 12869
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         12870 non-null  int64 
 1   age        12870 non-null  int64 
 2   job        12870 non-null  object
 3   marital    12870 non-null  object
 4   education  12870 non-null  object
 5   default    12870 non-null  object
 6   balance    12870 non-null  int64 
 7   housing    12870 non-null  object
 8   loan       12870 non-null  object
 9   contact    12870 non-null  object
 10  day        12870 non-null  int64 
 11  month      12870 non-null  object
 12  campaign   12870 non-null  int64 
 13  pdays      12870 non-null  int64 
 14  previous   12870 non-null  int64 
 15  poutcome   12870 non-null  object
 16  y          12870 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.7+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4291 entries, 0 to 4290
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         4291 non-null   int64 
 1   age        4291 non-null   int64 
 2   job        4291 non-null   object
 3   marital    4291 non-null   object
 4   education  4291 non-null   object
 5   default    4291 non-null   object
 6   balance    4291 non-null   int64 
 7   housing    4291 non-null   object
 8   loan       4291 non-null   object
 9   contact    4291 non-null   object
 10  day        4291 non-null   int64 
 11  month      4291 non-null   object
 12  campaign   4291 non-null   int64 
 13  pdays      4291 non-null   int64 
 14  previous   4291 non-null   int64 
 15  poutcome   4291 non-null   object
dtypes: int64(7), object(9)
memory usage: 536.5+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4291 entries, 0 to 4290
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       4291 non-null   int64  
 1   predict  4291 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 67.2 KB


None

In [20]:
from sklearn.model_selection import train_test_split
x = train.drop(columns =['ID', 'y'])
x_dummies = pd.get_dummies(x)
y = train['y']
x_train, x_test, y_train, y_test = train_test_split(x_dummies, y, stratify=y, random_state=10)

In [21]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
prediction = rf.predict_proba(x_test)

In [22]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, prediction[:,1])

0.7930218477378779

In [23]:
test_prediction = rf.predict_proba(pd.get_dummies(test.drop(columns=['ID'])))
submission['predict'] = test_prediction[:,1]
submission.head()

,ID,predict
0,53608,0.78
1,51055,0.90
2,52573,0.03
3,50458,0.08
4,52272,0.22


In [24]:
submission.to_csv('test.csv', index=False)